In [25]:
import joblib
import pandas as pd
import pandas_ta as ta
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import numpy as np
from sklearn.metrics import accuracy_score


# three models 
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier


# voting classifier of the ensemble
from sklearn.ensemble import VotingClassifier

In [16]:
loaded_lin_model = joblib.load('linear_model.pkl')
loaded_log_model = joblib.load('logistic_model.pkl')
loaded_rf_model = joblib.load('random_forest_classifier.pkl')

In [17]:
df_nflx_intraday = pd.read_csv('data_nflx_intraday.csv', index_col=0, parse_dates=True) # will be our working df
df = df_nflx_intraday.copy() # working df
df.columns = df.columns.str.lower()

In [18]:
df.head()

,open,high,low,close,volume
date,,,,,
2023-07-27 04:00:00,423.01,425.61,423.01,425.00,1150.0
2023-07-27 04:01:00,425.43,427.26,425.00,427.26,677.0
2023-07-27 04:02:00,427.19,427.19,426.00,426.70,341.0
2023-07-27 04:03:00,426.76,427.01,426.65,426.84,140.0
2023-07-27 04:04:00,426.84,427.64,426.84,427.49,117.0


In [19]:
df['price_increased'] = np.where(df['close'].diff() > 0, 1, 0)

In [20]:
df.head()

,open,high,low,close,volume,price_increased
date,,,,,,
2023-07-27 04:00:00,423.01,425.61,423.01,425.00,1150.0,0
2023-07-27 04:01:00,425.43,427.26,425.00,427.26,677.0,1
2023-07-27 04:02:00,427.19,427.19,426.00,426.70,341.0,0
2023-07-27 04:03:00,426.76,427.01,426.65,426.84,140.0,1
2023-07-27 04:04:00,426.84,427.64,426.84,427.49,117.0,1


In [21]:
# adding a few indicators
df['rsi'] = ta.rsi(df['close'], length=30)
df['ad'] = ta.ad(df['high'], df['low'], df['close'], df['volume']) # Accumulation/Distribution Line
bbands = ta.bbands(df['close'], length=10, mamode="ema") #  Returns: pd.DataFrame: lower, mid, upper, bandwidth, and percent columns.
df[['bbandsl', 'bbandsm', 'bbandsu','bbandsb', 'bbandsp']] = bbands[['BBL_10_2.0', 'BBM_10_2.0', 'BBU_10_2.0', 'BBB_10_2.0', 'BBP_10_2.0']]

# getting rid of nan
df.dropna(inplace=True)

In [22]:
# dividing X and y
X = df.drop(columns=['price_increased'])
y = df[['price_increased']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
loaded_log_model = joblib.load('logistic_model.pkl')
loaded_rf_model = joblib.load('random_forest_classifier.pkl')

In [24]:
voting_clf = VotingClassifier(
 estimators=[ ('log', loaded_log_model), ('rf', loaded_rf_model)],
 voting='hard')
voting_clf.fit(X_train, y_train)


c:\Users\ali\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\preprocessing\_label.py:97: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\ali\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\preprocessing\_label.py:132: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


VotingClassifier(estimators=[('log', LogisticRegression()),
                             ('rf', RandomForestClassifier(random_state=42))])

In [27]:
for clf in (loaded_log_model, loaded_rf_model, voting_clf):
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print(clf.__class__.__name__, accuracy_score(y_test, y_pred))

c:\Users\ali\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\ali\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


LogisticRegression 0.5094393592677345
RandomForestClassifier 0.7274027459954233


c:\Users\ali\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\preprocessing\_label.py:97: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\ali\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\preprocessing\_label.py:132: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


VotingClassifier 0.6101258581235698
